# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [29]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [30]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [31]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [32]:
ed = Website("https://substack.com/home/post/p-162210621?source=queue")
print("======================================================================")
ed.text

'Subscribe\nSign in\nShare this post\nM. V. Cunha\'s Substack\nTempus AI (TEM): Investment Thesis Explained 2.0\nCopy link\nFacebook\nEmail\nNotes\nMore\nTempus AI (TEM): Investment Thesis Explained 2.0\nM. V. Cunha\nApr 26, 2025\n23\nShare this post\nM. V. Cunha\'s Substack\nTempus AI (TEM): Investment Thesis Explained 2.0\nCopy link\nFacebook\nEmail\nNotes\nMore\n4\n1\nShare\nRoughly three months ago, I published my first Deep Dive on Tempus AI (TEM).\nSince then, several developments have emerged that could act as catalysts for the business.\nIt’s time to revisit the thesis and introduce a Valuation Model to assess whether TEM is an attractive investment from my perspective.\nSubscribe\nIntroduction\nTempus AI is a cutting-edge precision medicine company founded in 2015 by Eric Lefkofsky. The inspiration for Tempus arose from Lefkofsky’s personal life — his wife’s battle with breast cancer revealed how limited the role of technology was in shaping her care. Determined to change this

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [10]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [11]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [12]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [13]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/
https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/
https://edwarddonner.com/2025/01/23/ll

In [14]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [15]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/Qwen/Qwen3-235B-A22B',
 '/deepseek-ai/DeepSeek-Prover-V2-671B',
 '/nari-labs/Dia-1.6B',
 '/Qwen/Qwen3-30B-A3B',
 '/Qwen/Qwen3-32B',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/nari-labs/Dia-1.6B',
 '/spaces/Qwen/Qwen3-Demo',
 '/spaces/stepfun-ai/Step1X-Edit',
 '/spaces/tencent/Hunyuan3D-2',
 '/spaces',
 '/datasets/nvidia/OpenMathReasoning',
 '/datasets/OpenGVLab/InternVL-Data',
 '/datasets/nvidia/OpenCodeReasoning',
 '/datasets/nvidia/Nemotron-CrossThink',
 '/datasets/Eureka-Lab/PHYBench',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/transformers',
 '/docs/diffusers',
 '/docs/safetens

In [16]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'community page', 'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [17]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [18]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
Qwen/Qwen3-235B-A22B
Updated
3 days ago
•
2

In [19]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [20]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [23]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nQwen/Qwen3-235B-A22B\nUpdated\n3 days ago\n•\n28.6k\n•\n622\ndeepseek-ai/DeepSeek-Prover-V2-671B\nUpdated\n3 days ago\n•\n1.71k\n•\n606\nnari-labs/Dia-1.6B\nUpdated\n6 days ago\n•\n108k\n•\n1.75k\nQwen/Qwen3-30B-A3B\nUpdated\n3 days ago\n•\n39.8k\n•\n419\nQwen/Qwen3-32B\nUpdated\n4 days ago\n•\n73k\n•\n258\nBrowse 1M+ models\nSpaces\nRunning\n5.81k\n5.81k\nDeepSite\n🐳\nGenerate any application

In [24]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [25]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'homepage', 'url': 'https://huggingface.co/'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Brochure

## Welcome to Hugging Face
The AI community building the future. Hugging Face is the leading platform for collaboration among machine learning (ML) professionals and enthusiasts. Here, we foster creativity and innovation in developing models, datasets, and applications that make AI accessible and beneficial for all.

---

## Why Choose Hugging Face?

### Community Collaboration
- **Platform for Collaboration**: Engage with a vibrant community of over 50,000 organizations, including industry giants like Google, Amazon, and Microsoft.  
- **Open Source Tools**: Contribute to or utilize our wide array of open-source ML tools including Transformers, Diffusers, and Tokenizers to accelerate your projects.

### Diverse Offerings
- **1M+ Models**: Access an extensive range of machine learning models, including trending innovations like Qwen/Qwen3 and DeepSeek-Prover.
- **Datasets Galore**: Browse over 250,000 datasets to fuel your machine learning projects.
- **Innovative Applications**: Explore AI applications tailored for text, image, video, audio, and even 3D.

### Scalable Solutions
- **Enterprise Grade**: Our paid Compute and Enterprise solutions provide you with optimized inference endpoints and dedicated support—ensuring your organization has the resources and tools necessary to lead in AI development.

### Careers at Hugging Face
Join a passionate team committed to shaping the future of AI. We’re always looking for enthusiastic individuals who share our vision. Embrace a culture that prioritizes learning, innovation, and collaboration.
- **Flexible Work Environment**: We support remote and flexible work arrangements to promote work-life harmony.
- **Growth Opportunities**: With numerous resources available for personal and professional development, your career will thrive with us!

---

## Join Us Today!
Whether you are a developer looking to enhance your portfolio, a company seeking to scale AI efforts, or a talented individual ready to make an impact, Hugging Face is the place for you.
- **Explore AI Apps**
- **Browse 1M+ Models**
- **Sign Up for Free**

---

## Connect with Us
Stay updated with the latest advancements and discussions in AI by following us on social media or visiting our community forums.
- **GitHub** | **Twitter** | **LinkedIn** | **Discord**

---

At Hugging Face, we believe in the power of AI to transform industries and enhance lives. Join us in building the future together!

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [26]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [27]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}


# Hugging Face Company Brochure

Welcome to Hugging Face, the AI community dedicated to building the future of machine learning. Our platform provides a collaborative environment where developers, researchers, and organizations come together to create, share, and innovate on machine learning models, datasets, and applications.

## Company Overview

### The AI Community
Hugging Face is at the forefront of the AI community, enabling the collaboration of over 50,000 organizations, from prominent names like Google, Microsoft, and Amazon to innovative startups and research labs. Our platform not only democratizes access to advanced machine learning technologies but also fosters a spirit of community and knowledge-sharing among its users.

### Our Offerings
- **Models**: Explore over 1 million models across various applications and specialties.
- **Datasets**: Access 250,000+ datasets to enhance your machine learning projects.
- **Spaces**: Develop, run, and share machine learning applications effortlessly with our easy-to-use interface.
- **Enterprise Solutions**: Tailored services providing enterprise-grade security, access control, and dedicated support.

## Company Culture

At Hugging Face, we embrace an open-source philosophy, fostering transparency, community contribution, and collaboration. The spirit of innovation thrives here, with a commitment to ethical AI and a desire to shape the future responsibly. We aim to build a workplace where creativity and teamwork are celebrated, leading to impactful advancements in AI technology.

## Careers at Hugging Face

Hugging Face is always on the lookout for passionate individuals to join our team. We believe in nurturing talent and providing opportunities for professional growth. Our team members enjoy a supportive environment with the flexibility to explore their interests in AI and ML.

### Current Openings
We regularly update our career page with various openings. Whether you’re a developer, data scientist, or product manager, there may be a role for you to make a difference in the AI landscape. 

### Employee Benefits
- Competitive salary and benefits
- Flexible work arrangements
- Opportunities for learning and development
- A vibrant community of like-minded professionals

## Join Us in Shaping the Future

Hugging Face invites you to be part of a movement that is reshaping the world of AI and machine learning. Whether you’re a customer, investor, or prospective recruit, your journey starts here. 

### Get Involved
- **Explore AI Apps**: Check out our platform for a wide range of applications and models.
- **Connect with Us**: Follow us on our social platforms like GitHub, Twitter, LinkedIn, and Discord to stay updated.
- **Sign Up Today**: Join our community and start collaborating.

---

Thank you for your interest in Hugging Face. Together, we’re building the future of AI. For more information, visit our [website](https://huggingface.co).

In [28]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'home page', 'url': 'https://huggingface.co/'}, {'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}



# Hugging Face Brochure

## Welcome to Hugging Face!

### The AI Community Building the Future
Hugging Face is a vibrant platform where machine learning enthusiasts and developers come together to collaborate on innovative models, datasets, and AI applications. We pride ourselves on fostering an open and inclusive AI community that is reshaping the future of technology.

## Our Offerings

### **Models**
Explore our extensive repository of over 1 million models, including:
- **Qwen/Qwen3-235B-A22B**: Top trending model this week with 28.6k downloads!
- **DeepSeek-Prover-V2-671B**: Updated regularly, pushing boundaries in AI.
- **Dia-1.6B**: A powerful tool for generating realistic dialogues.

### **Datasets**
Access over 250k datasets to help fuel your machine learning projects, including specialized collections like **OpenCodeReasoning** and **Nemotron-CrossThink**.

### **Spaces**
Experiment with running applications directly in our dedicated Spaces feature, where you can generate AI applications and showcase your work to the community.

### **Enterprise Solutions**
We offer tailored enterprise solutions with:
- **Advanced security and support**
- **Priority access and dedicated resources**
- **Affordable pricing starting at $20/user/month**

## Who Uses Us?

Join a network of over 50,000 organizations leveraging Hugging Face, including:
- **Meta, Google, Microsoft, and Amazon**
- Non-profits and startups utilizing our models to innovate across industries.

## Company Culture

At Hugging Face, we believe in:
- **Collaboration**: Work together to innovate and advance the field of machine learning.
- **Openness**: Our resources and models are available for everyone, ensuring inclusivity in AI development.
- **Growth**: We support individual growth by encouraging team members to contribute to open-source projects and build their ML profiles.

## Careers at Hugging Face

Join our team of visionaries and technologists. We are constantly on the lookout for:
- **Machine Learning Engineers**
- **Data Scientists**
- **Product Managers**

### Why Work with Us?
- Contribute to impactful AI projects with a dedicated team.
- Thrive in a flexible work environment that encourages creativity and innovation.
- Benefit from continuous learning opportunities and resources.

## Get Involved!

### **Join Us Today!**
Discover how you can be a part of shaping the future of AI by signing up at [Hugging Face](https://huggingface.co).

### **Connect with Us**
Stay updated and join the conversation on our social media platforms:
- [Twitter](https://twitter.com/huggingface)
- [LinkedIn](https://linkedin.com/company/huggingface)
- [GitHub](https://github.com/huggingface)

Let’s build the future together!



<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>